In [2]:
import numpy as np
import matplotlib.pyplot as plt

from rough_bergomi.models import RoughBergomiModel, GBMModel, HestonModel
from rough_bergomi.utils.utils import bs_delta, bsinv, bs  # Black-Scholes inversion for IV calculation
import seaborn as sns
import yfinance as yf

from rough_bergomi.hedging.deep_hedger import DeepHedgerTF

In [4]:
hidden_sizes = [16, 64, 128]
n_layers = [1, 2, 3]
optimizers = ['adam', 'rmsprop']

model = RoughBergomiModel()
results = []

for opt in optimizers:
    for h in hidden_sizes:
        for l in n_layers:
            print(f"Running: {l} GRU layers, {h} units, {opt.upper()}")
            hedger = DeepHedgerTF(
                model=model,
                M=10000,
                N=100,
                T=30,
                lambda_tc=0.0  # no transaction cost
            )
            hedger.prepare_data()
            hedger.build_and_compile_model(lr=0.001, loss_type='mshe',
                                           gru_layers=l,
                                           hidden_size=h,
                                           optimizer=opt)
            hedger.train_model(EPOCHS=30, BATCH_SIZE=256)

            hedging_error = hedger.evaluate_model()
            std_error = np.std(hedging_error)
            mean_error = np.mean(hedging_error)
            cvar_1_error = np.mean(np.sort(hedging_error)[:int(0.01 * len(hedging_error))])

            results.append((l, h, opt, mean_error, std_error, cvar_1_error))

Running: 1 GRU layers, 16 units, ADAM
Mean hedging error: -0.361181238439193
Std of hedging error: 19.08470447220171
Running: 2 GRU layers, 16 units, ADAM
Mean hedging error: -0.3527849987826297
Std of hedging error: 18.94726799454756
Running: 3 GRU layers, 16 units, ADAM
Mean hedging error: -0.3743066664844242
Std of hedging error: 20.435644527710657
Running: 1 GRU layers, 64 units, ADAM
Mean hedging error: -0.3772909823969879
Std of hedging error: 20.331418316306742
Running: 2 GRU layers, 64 units, ADAM
Mean hedging error: -0.3200305698222683
Std of hedging error: 16.967922789267842
Running: 3 GRU layers, 64 units, ADAM
Mean hedging error: -0.47056938625133005
Std of hedging error: 28.948204148549465
Running: 1 GRU layers, 128 units, ADAM
Mean hedging error: -0.2728692730597881
Std of hedging error: 17.117653187246383
Running: 2 GRU layers, 128 units, ADAM
Mean hedging error: -0.47293989348690385
Std of hedging error: 27.046371626021468
Running: 3 GRU layers, 128 units, ADAM
Mean hed